In [ ]:
# item子集
import numpy as np
input_dir = "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_init_change_chuangkou_90_180/"
tmp_embedding = np.load(input_dir + "embeddings.npy")
print(tmp_embedding.shape)


In [ ]:
import numpy as np
import os

# 定义要处理的文件夹列表
# input_dir_np10 = "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_category_propensity_new/"
# np10 = np.load(input_dir_np10 + "embeddings.npy")

# input_dir_np11 = "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_category_propensity_new_add_novelty/"
# np11 = np.load(input_dir_np11 + "embeddings.npy")

# input_dir_np12 = "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_category_skus_onehot*2_L2_normal/"
# np12 = np.load(input_dir_np12 + "embeddings.npy")
data_dirs = [
    "/data_sdc/mhwang/RecSys_sub/Z_yichen/yichen_price_zscore_feats",
    "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_1_ewa_chuangkoulada",
    "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_init_change_chuangkou_90_180",
    "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_new",
    "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_price_new",
    "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_category_propensity_new"
    "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_category_propensity_new_add_novelty",
    "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_category_skus_onehot*2_L2_normal"
    
]

for data_dir in data_dirs:
    # 检查文件夹是否存在
    if not os.path.exists(data_dir):
        print(f"文件夹 {data_dir} 不存在，跳过")
        continue
    
    # 构建文件路径
    client_ids_path = os.path.join(data_dir, "client_ids.npy")
    embeddings_path = os.path.join(data_dir, "embeddings.npy")
    
    # 检查文件是否存在
    if not (os.path.exists(client_ids_path) and os.path.exists(embeddings_path)):
        print(f"文件夹 {data_dir} 中缺少必要文件，跳过")
        continue
    
    # 加载数据
    client_ids = np.load(client_ids_path)
    embeddings = np.load(embeddings_path)
    
    # 获取 client_ids 的排序索引
    sorted_indices = np.argsort(client_ids)
    
    # 按排序索引重新排列 client_ids 和 embeddings
    sorted_client_ids = client_ids[sorted_indices]
    sorted_embeddings = embeddings[sorted_indices]
    sorted_embeddings = sorted_embeddings.astype(np.float16)

    # 保存排序后的数据
    np.save(client_ids_path, sorted_client_ids)
    np.save(embeddings_path, sorted_embeddings)
    
    print(f'文件夹 {data_dir} 中的 client_ids 和 embeddings 已按 client_ids 下标从小到大排序并保存')

文件夹 /data_sdc/mhwang/RecSys_sub/Z_yichen/yichen_price_zscore_feats 中的 client_ids 和 embeddings 已按 client_ids 下标从小到大排序并保存
文件夹 /data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_1_ewa_chuangkoulada 中的 client_ids 和 embeddings 已按 client_ids 下标从小到大排序并保存
文件夹 /data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_init_change_chuangkou_90_180 中的 client_ids 和 embeddings 已按 client_ids 下标从小到大排序并保存
文件夹 /data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_new 中的 client_ids 和 embeddings 已按 client_ids 下标从小到大排序并保存
文件夹 /data_sdc/mhwang/recsys_code/submit_file_new/dlrm_price_new 中的 client_ids 和 embeddings 已按 client_ids 下标从小到大排序并保存
文件夹 /data_sdc/mhwang/recsys_code/submit_file_new/dlrm_category_propensity_new/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_category_propensity_new_add_novelty 不存在，跳过
文件夹  不存在，跳过


In [ ]:
import numpy as np

# Define L2 normalization function with enhanced stability for float16
def l2_normalize(emb, eps=1e-12, clip_value=1e3, batch_size=10000):
    """
    L2 normalize embeddings in batches with clipping to prevent overflow, optimized for float16 inputs.
    Args:
        emb: Input embedding array (n_samples, dim).
        eps: Small value to avoid division by zero.
        clip_value: Clip values to [-clip_value, clip_value].
        batch_size: Process in batches to save memory.
    Returns:
        Normalized embedding array in float16.
    """
    # Convert to float32 for stable computation
    emb = emb.astype(np.float32)
    result = np.zeros_like(emb, dtype=np.float32)
    for i in range(0, len(emb), batch_size):
        batch = np.clip(emb[i:i+batch_size], -clip_value, clip_value)
        # Compute norm with float32 precision
        norm = np.sqrt(np.sum(batch**2, axis=1, keepdims=True))
        norm = np.maximum(norm, eps)  # Avoid division by zero
        result[i:i+batch_size] = batch / norm
        # Replace inf/nan with zeros
        result[i:i+batch_size] = np.nan_to_num(result[i:i+batch_size], nan=0.0, posinf=0.0, neginf=0.0)
    # Clip to float16-safe range and convert to float16
    result = np.clip(result, -65504, 65504)
    return result.astype(np.float16)

# Load embeddings (assuming float16 inputs)

input_dir_np5 = "/data_sdc/mhwang/RecSys_sub/Z_yichen/yichen_price_zscore_feats/"
np5 = np.load(input_dir_np5 + "embeddings.npy")

input_dir_np6 = "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_1_ewa_chuangkoulada/"
np6 = np.load(input_dir_np6 + "embeddings.npy")

input_dir_np7 = "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_init_change_chuangkou_90_180/"
np7 = np.load(input_dir_np7 + "embeddings.npy")

input_dir_np8 = "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_new/"
np8 = np.load(input_dir_np8 + "embeddings.npy")

input_dir_np9 = "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_price_new/"
np9 = np.load(input_dir_np9 + "embeddings.npy")

input_dir_np10 = "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_category_propensity_new/"
np10 = np.load(input_dir_np10 + "embeddings.npy")

input_dir_np11 = "/data_sdc/mhwang/recsys_code/submit_file_new/dlrm_churn_category_propensity_new_add_novelty/"
np11 = np.load(input_dir_np11 + "embeddings.npy")

input_dir_np12 = "/data_sdc/mhwang/RecSys_sub/Z_new_data/ruizhi_category_skus_onehot*2_L2_normal/"
np12 = np.load(input_dir_np12 + "embeddings.npy")

# Detailed inspection of input embeddings
for name, emb in [("np5", np5), ("np6", np6), ("np7", np7), ("np8", np8), ("np9", np9), ("np10", np10), ("np11", np11), ("np12", np12)]:
    # Convert to float32 for stable norm calculation
    emb_float32 = emb.astype(np.float32)
    norms = np.sqrt(np.sum(emb_float32**2, axis=1))
    finite_norms = norms[np.isfinite(norms)]
    print(f"\n{name} statistics:")
    print(f"  Max absolute value: {np.abs(emb_float32).max():.3e}, Min: {np.abs(emb_float32).min():.3e}")
    print(f"  Max norm: {np.max(norms) if len(norms) > 0 else 'N/A':.3e}")
    print(f"  Mean norm: {np.mean(finite_norms) if len(finite_norms) > 0 else 'N/A':.3e}")
    print(f"  Norm std: {np.std(finite_norms) if len(finite_norms) > 0 else 'N/A':.3e}")
    print(f"  Contains inf/nan: {np.any(np.isinf(emb)) or np.any(np.isnan(emb))}")
    print(f"  Number of inf norms: {np.sum(np.isinf(norms))}")
    print(f"  Number of zero norms: {np.sum(norms == 0)}")

# L2 normalize each embedding
np5_norm = l2_normalize(np5)
np6_norm = l2_normalize(np6)
np7_norm = np7
np8_norm = l2_normalize(np8)
np9_norm = l2_normalize(np9)
np10_norm = l2_normalize(np10)
np11_norm = l2_normalize(np11)
np12_norm = l2_normalize(np12)

# Verify normalized norms
for name, emb in [("np5_norm", np5_norm), ("np6_norm", np6_norm), ("np7_norm", np7_norm), ("np8_norm", np8_norm),
                  ("np9_norm", np9_norm), ("np10_norm", np10_norm), ("np11_norm", np11_norm), ("np12_norm", np12_norm)]:
    norms = np.sqrt(np.sum(emb.astype(np.float32)**2, axis=1))
    finite_norms = norms[np.isfinite(norms)]
    print(f"{name} mean norm after normalization: {np.mean(finite_norms):.3e}")

# Concatenate normalized embeddings
combined_emb = np.concatenate((np6_norm, np7_norm, np5_norm, np11_norm, np9_norm, np10_norm, np12_norm), axis=1)

# Ensure float16 safety
combined_emb = np.clip(combined_emb, -65504, 65504).astype(np.float16)

# Final inspection
print("\nFinal combined_emb statistics:")
print(f"  Max absolute value: {np.abs(combined_emb).max():.3e}, Min: {np.abs(combined_emb).min():.3e}")
print(f"  Contains inf/nan: {np.any(np.isinf(combined_emb)) or np.any(np.isnan(combined_emb))}")

# Save result
save_dir = "/data_sdc/mhwang/RecSys_sub/Z_new_data_train/concat_all/concat_84_v63_ewa_ewa_have_normal/"
np.save(save_dir + "embeddings_normalized.npy", combined_emb)
print("Shape of normalized concatenated embeddings:", combined_emb.shape)